In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

## Imports and Library Setup

In [ ]:
import { buildParser } from '@lezer/generator';
import { Tree, TreeCursor } from '@lezer/common';
import { LRParser } from '@lezer/lr';
import { RecursiveSet, RecursiveMap, Tuple } from "recursive-set";

# Symbolischer Taschenrechner mit Lezer \& RecursiveSet

In diesem Notebook implementieren wir einen **symbolischen Taschenrechner**, der Zuweisungen und arithmetische Ausdrücke parsen, analysieren und auswerten kann.

Die Besonderheit dieser Implementierung liegt in der Verwendung von **Value Semantics** für den Abstract Syntax Tree (AST) und das Environment. Anstatt gewöhnlicher JavaScript-Objekte nutzen wir die Bibliothek `recursive-set`, um strukturelle Gleichheit und hash-basiertes Lookup zu ermöglichen.

## 1. Architektur \& Konzepte

Der Interpreter durchläuft eine Pipeline aus vier Stufen:

1. **Grammatik-Definition:** Wir definieren die Sprache deklarativ mit **Lezer**.
2. **Runtime Parsing:** `buildParser` generiert daraus einen effizienten LR-Parser.
3. **AST-Transformation (Tuple-basiert):** Der flache Lezer-Tree wird in einen typisierten AST umgewandelt. Hierbei erben alle Knoten von der Klasse `Tuple`.
4. **Semantische Analyse \& Evaluation:** Wir nutzen `RecursiveSet` zur Variablenanalyse und `RecursiveMap` als Speicher für Variablen.

### Warum `Tuple` und `RecursiveSet`?

In klassischen ASTs sind zwei Knoten `new BinOp("+", 1, 2)` und `new BinOp("+", 1, 2)` **nicht** gleich (Referenzgleichheit). Durch das Erben von `Tuple` erhalten wir **Wertegleichheit (Structural Equality)**:

* **Identität:** Zwei AST-Knoten mit gleichem Inhalt sind logisch identisch.
* **Hashing:** Wir können AST-Knoten als Keys in Maps oder Elemente in Sets verwenden.
* **Debugging:** `Tuple` liefert automatisch eine saubere String-Repräsentation (z.B. `(10 * 2)`).


## 2. Domain Types (Der AST)

Wir definieren die Knoten unserer Sprache als Klassen, die von `Tuple` erben. Dies garantiert Typ-Sicherheit und Unveränderlichkeit.

* `Num`: Repräsentiert eine Zahl.
* `Var`: Repräsentiert eine Variable.
* `BinOp`: Repräsentiert binäre Operationen (`+`, `-`, `*`, `/`).
* `Assign`: Repräsentiert eine Zuweisung (`x := ...`).

In [ ]:
type Operator = "+" | "-" | "*" | "/";

class Num extends Tuple<["num", number]> {
    constructor(val: number) {
        super("num", val);
    }
    get value(): number {
        return this.get(1);
    }

    toString() {
        return String(this.value);
    }
}

class Var extends Tuple<["var", string]> {
    constructor(name: string) {
        super("var", name);
    }
    get name(): string {
        return this.get(1);
    }

    toString() {
        return this.name;
    }
}

class BinOp<L extends Expr = Expr, R extends Expr = Expr> extends Tuple<
    [Operator, L, R]
> {
    constructor(op: Operator, left: L, right: R) {
        super(op, left, right);
    }

    get op(): Operator {
        return this.get(0);
    }
    get left(): L {
        return this.get(1);
    }
    get right(): R {
        return this.get(2);
    }

    toString() {
        return `(${this.left} ${this.op} ${this.right})`;
    }
}

class Assign extends Tuple<["assign", string, Expr]> {
    constructor(name: string, expr: Expr) {
        super("assign", name, expr);
    }

    get name(): string {
        return this.get(1);
    }
    get expr(): Expr {
        return this.get(2);
    }

    toString() {
        return `${this.name} := ${this.expr};`;
    }
}

class ExprStmnt extends Tuple<["expr_stmnt", Expr]> {
    constructor(expr: Expr) {
        super("expr_stmnt", expr);
    }

    get expr(): Expr {
        return this.get(1);
    }

    toString() {
        return `${this.expr};`;
    }
}

type Expr = Num | Var | BinOp;
type Stmnt = Assign | ExprStmnt;
type Env = RecursiveMap<string, number>;

function isOperator(s: string): s is Operator {
    return ["+", "-", "*", "/"].includes(s);
}

function isExpr(node: any): node is Expr {
    return node instanceof Num || node instanceof Var || node instanceof BinOp;
}

function ensureExpr(node: any): Expr {
    if (isExpr(node)) return node;
    throw new Error(
        `Erwarte Expr, erhielt: ${node?.constructor?.name ?? node}`,
    );
}

## 3. Grammatik (Lezer)

Die Grammatik definiert die Syntaxregeln. Wir verwenden Großbuchstaben für Regeln wie `Expr` oder `Factor`, um sicherzustellen, dass Lezer eigenständige Knoten im Parse-Tree erzeugt. Dies erleichtert die spätere Navigation.

* **Links-Rekursion:** Wird durch Wiederholungen (`Product ((Plus | Minus) Product)*`) aufgelöst.
* **Vorrangregeln:** Punkt-vor-Strich wird durch die Schachtelung von `Expr` (Strichrechnung) und `Product` (Punktrechnung) erzwungen.

In [ ]:
const grammarDefinition = `
@top Program { statement+ }

statement {
  Assignment { Identifier ":=" Expr ";" } |
  ExpressionStatement { Expr ";" }
}

Expr {
  Product ((Plus | Minus) Product)*
}

Product {
  Factor ((Mul | Div) Factor)*
}

Factor {
  ParenExpr { "(" Expr ")" } |
  UnaryExpr { (Plus | Minus) Factor } |
  Number |
  Identifier
}

@tokens {
  Number { (("0" | $[1-9] $[0-9]*) ("." $[0-9]*)?) }
  Identifier { $[a-zA-Z] $[a-zA-Z0-9_]* }
  Plus { "+" }
  Minus { "-" }
  Mul { "*" }
  Div { "/" }
  space { $[ \t\r]+ }
  "(" ")" ";" ":="
}

@skip { space }
`;

const parser: LRParser = buildParser(grammarDefinition);

### Helper Function `tokenizeCalc`

In Lezer, tokenization is tightly integrated with the parsing process. Instead of producing a separate list of tokens upfront, the parser directly builds a tree where the leaves represent the tokens.

To inspect how the input is being tokenized for debugging purposes, we implement `tokenizeCalc`. This function:

1. **Parses** the input to build the full syntax tree.
2. **Traverses** the tree using a `TreeCursor`.
3. **Filters** for leaf nodes (nodes without children), which correspond to the atomic tokens (e.g., `Number`, `Plus`, `Identifier`).

**Input:**

* `input`: A source code string (`string`).

**Output:**

* A list of strings (`string[]`), where each entry describes the token type and its raw content (e.g., `"Number('42')"`).

**Note:** This function is a utility for inspection. The actual AST transformation logic operates on the tree structure directly, rather than a flat list of tokens.

In [ ]:
function tokenizeCalc(input: string): string[] {
    const tree = parser.parse(input);
    const cursor = tree.cursor();
    const tokens: string[] = [];
    const structuralNodes = new Set([
        "Program",
        "statement",
        "Assignment",
        "ExpressionStatement",
        "Expr",
        "Product",
        "Factor",
        "UnaryExpr",
        "ParenExpr",
    ]);
    do {
        if (!structuralNodes.has(cursor.name)) {
            const content = input.slice(cursor.from, cursor.to);
            let name = cursor.name;
            if (name === ":=") name = "Assign";
            if (name === ";") name = "Semicolon";

            tokens.push(`${name}('${content}')`);
        }
    } while (cursor.next());
    return tokens;
}


Example: 

In [ ]:
console.log(tokenizeCalc("x := 3 + 4 * 2;"));

## 4. Parser-Logik (`lezerToAST`)

Die Funktion `lezerToAST` transformiert den effizienten, aber flachen Lezer-Tree (`TreeCursor`) in unsere reichhaltige `Tuple`-Struktur.

**Wichtige Features:**

* **Robuste Navigation:** Wir suchen Kind-Knoten dynamisch anhand ihres Namens, anstatt uns auf feste Index-Positionen zu verlassen. Das macht den Parser resistent gegen kleine Grammatikänderungen.
* **Type Guards:** Wir verwenden `ensureExpr` und `isExpr`, um zur Laufzeit sicherzustellen, dass wir keine Statements an Stellen erhalten, wo Ausdrücke erwartet werden.
* **Fehlerbehandlung:** Syntaxfehler (Knoten `⚠`) werden sofort abgefangen.

In [ ]:
function lezerToAST(cursor: TreeCursor, source: string): Stmnt | Expr {
    const name = cursor.name;
    const text = source.slice(cursor.from, cursor.to);

    if (name === "⚠") throw new Error(`Syntax-Fehler nahe: '${text}'`);

    switch (name) {
        case "Assignment": {
            let varName: string | null = null;
            let exprNode: Expr | null = null;

            if (cursor.firstChild()) {
                do {
                    if (cursor.name === "Identifier") {
                        varName = source.slice(cursor.from, cursor.to);
                    } else if (cursor.name === "Expr") {
                        exprNode = ensureExpr(lezerToAST(cursor, source));
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }

            if (!varName) throw new Error("Assignment: Identifier fehlt");
            if (!exprNode) throw new Error("Assignment: Expression fehlt");

            return new Assign(varName, exprNode);
        }

        case "ExpressionStatement": {
            let exprNode: Expr | null = null;
            if (cursor.firstChild()) {
                do {
                    if (cursor.name === "Expr") {
                        exprNode = ensureExpr(lezerToAST(cursor, source));
                        break;
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }

            if (!exprNode) throw new Error("Leeres Statement");
            return new ExprStmnt(exprNode);
        }

        case "Expr":
            return parseLeftAssociative(cursor, source, ["+", "-"]);

        case "Product":
            return parseLeftAssociative(cursor, source, ["*", "/"]);

        case "UnaryExpr": {
            let op: Operator = "+";
            let factor: Expr | null = null;

            if (cursor.firstChild()) {
                do {
                    const t = source.slice(cursor.from, cursor.to);
                    if (isOperator(t)) {
                        op = t;
                    } else {
                        factor = ensureExpr(lezerToAST(cursor, source));
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }

            if (!factor) throw new Error("UnaryExpr ohne Wert");

            if (op === "-") {
                return new BinOp("*", new Num(-1), factor);
            }
            return factor;
        }

        case "ParenExpr": {
            let inner: Expr | null = null;
            if (cursor.firstChild()) {
                do {
                    if (cursor.name === "Expr") {
                        inner = ensureExpr(lezerToAST(cursor, source));
                        break;
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }
            return inner || new Num(0);
        }

        case "Number":
            return new Num(parseFloat(text));

        case "Identifier":
            return new Var(text);

        default:
            if (cursor.firstChild()) {
                const result = lezerToAST(cursor, source);
                cursor.parent();
                return result;
            }
            throw new Error(`Unerwarteter AST-Knoten: ${name}`);
    }
}

function parseLeftAssociative(
    cursor: TreeCursor,
    source: string,
    allowedOps: string[],
): Expr {
    const operands: Expr[] = [];
    const operators: Operator[] = [];

    if (cursor.firstChild()) {
        do {
            const text = source.slice(cursor.from, cursor.to);
            if (allowedOps.includes(text)) {
                if (isOperator(text)) operators.push(text);
            } else {
                operands.push(ensureExpr(lezerToAST(cursor, source)));
            }
        } while (cursor.nextSibling());
        cursor.parent();
    }

    if (operands.length === 0) return new Num(0);

    let result = operands[0];
    for (let i = 0; i < operators.length; i++) {
        const op = operators[i];
        const right = operands[i + 1];
        if (!right) throw new Error(`Operator '${op}' fehlt rechter Operand.`);

        result = new BinOp(op, result, right);
    }

    return result;
}

## 5. Analyse und Umgebung

Anstatt eines einfachen `Record<string, number>` verwenden wir `RecursiveMap` für das Environment (`Env`).

**Vorteile:**

1. **Effizienz:** Optimiertes Hashing für String-Keys.
2. **Output:** `env.toString()` liefert eine deterministische, sortierte Ausgabe des Speicherzustands.

Zusätzlich implementieren wir `collectVars`, welche alle verwendeten Variablen eines Ausdrucks in einem `RecursiveSet<string>` sammelt. Dies demonstriert, wie einfach Mengenoperationen mit der Library sind.

In [ ]:
function collectVars(node: Stmnt | Expr): RecursiveSet<string> {
    const vars = new RecursiveSet<string>();

    function visit(n: Stmnt | Expr): void {
        if (n instanceof Var) {
            vars.add(n.name);
        } else if (n instanceof Assign) {
            vars.add(n.name);
            visit(n.expr);
        } else if (n instanceof ExprStmnt) {
            visit(n.expr);
        } else if (n instanceof BinOp) {
            visit(n.left);
            visit(n.right);
        }
    }

    visit(node);
    return vars;
}

## 5. Evaluation (Operational Semantics)

The evaluation phase defines the semantic meaning of our AST nodes. It uses the `RecursiveMap` environment to store and retrieve variable values efficiently.

### Expression Evaluation (`evalExpr`)

This recursive function computes the numerical value of an expression tree (`Expr`). It uses `instanceof` checks to distinguish between node types, leveraging the TypeScript type system for safety.

* **`Num`**: Returns the raw numeric value directly.
* **`Var`**: Looks up the variable name in the `env` map. The `env.get()` method is efficient (O(1) average) and type-safe. If a variable is missing, a runtime error is thrown.
* **`BinOp`**: Recursively evaluates the left and right operands first (Depth-First Traversal) and then applies the corresponding mathematical operation. This naturally respects the operator precedence encoded in the AST structure.


### Statement Execution (`execStmnt`)

Statements are the top-level units of execution that can cause side effects (modifying the environment).

* **`Assign`**: First evaluates the right-hand side expression. Then, it updates the environment using `env.set(name, value)`. The result of the assignment is the value itself.
* **`ExprStmnt`**: Evaluates the expression purely for its result (e.g., as a calculator output) without modifying the environment.


In [ ]:
function evalExpr(e: Expr, env: Env): number {
    if (e instanceof Num) {
        return e.value;
    } else if (e instanceof Var) {
        const val = env.get(e.name);
        if (val === undefined)
            throw new Error(`Variable '${e.name}' ist nicht definiert.`);
        return val;
    } else if (e instanceof BinOp) {
        const l = evalExpr(e.left, env);
        const r = evalExpr(e.right, env);
        switch (e.op) {
            case "+":
                return l + r;
            case "-":
                return l - r;
            case "*":
                return l * r;
            case "/":
                return l / r;
        }
    }
    throw new Error("Unbekannter Ausdruckstyp");
}

function execStmnt(s: Stmnt, env: Env): number | undefined {
    if (s instanceof Assign) {
        const val = evalExpr(s.expr, env);
        env.set(s.name, val);
        return val;
    } else if (s instanceof ExprStmnt) {
        return evalExpr(s.expr, env);
    }
    return undefined;
}

## 6. Main Execution Loop

The `main` block ties the parsing, analysis, and evaluation phases together into a Read-Eval-Print Loop (REPL) simulation.

1. **Environment Initialization:** We create a single, persistent `RecursiveMap<string, number>` that holds the state across all statements.
2. **Input Processing:** The raw input string is split into individual lines to simulate sequential execution.
3. **The Loop:**
    * **Parse:** `parseCalc` converts the input string into a typed `Stmnt` AST. It ensures that the result is a valid statement and not just a raw expression.
    * **Analyze:** `collectVars` is called to identify all variables referenced in the current line (demonstrating static analysis).
    * **Execute:** `execStmnt` runs the statement against the current environment.
    * **Output:** The results are logged. Thanks to `Tuple` and `RecursiveSet`, the `toString()` output is automatically formatted and readable (e.g., sets are printed as `{x, y}`).

In [ ]:
function parseCalc(input: string): Stmnt {
    const tree = parser.parse(input);
    const cursor = tree.cursor();

    if (!cursor.firstChild()) throw new Error("Leeres Programm");
    if (cursor.name === "Program") {
        if (!cursor.firstChild()) throw new Error("Leeres Programm (Body)");
    }

    const ast = lezerToAST(cursor, input);
    if (isExpr(ast)) throw new Error("Erwarte Statement");
    return ast as Stmnt;
}

Eingabe Parameter für den Taschenrechner als `inputProgram` festlegen:

In [ ]:
const inputProgram = `
x := 10 + 5 * 2;
y := (x - 5) / 3;
z := -y * 2;
z + 100;
`;

### Final State Inspection

After processing all lines, we print the final environment using `env.toString()`. Unlike standard JavaScript objects, `RecursiveMap` produces a deterministic, sorted output, which is invaluable for verification and debugging.

In [ ]:
const env = new RecursiveMap<string, number>();

const lines = inputProgram
    .split("\n")
    .map((l) => l.trim())
    .filter((l) => l.length > 0);

console.log("--- Calculation Start ---");

for (const line of lines) {
    try {
        const stmnt = parseCalc(line);
        const vars = collectVars(stmnt);
        const res = execStmnt(stmnt, env);
        console.log(
            `AST: ${stmnt.toString().padEnd(30)} | Vars: ${vars} | Result: ${res}`,
        );
    } catch (e) {
        console.error(
            `Error in '${line}':`,
            e instanceof Error ? e.message : e,
        );
    }
}

console.log("\n--- Final Environment ---");
console.log(env.toString());
